In [22]:
import numpy as np

In [28]:
with open("labels.txt", "r") as fp:
    labels = np.array([elem for idx, elem in enumerate(fp.read().split('"')) if idx % 2 == 1])
    
with open("distance_matrix.txt", "r") as fp:
    distance_matrix = np.zeros((labels.shape[0], labels.shape[0]))
    for idx, line in enumerate(fp.read().splitlines()):
        line_data = np.array(line.replace(' ', '')[1:-1].split(','), dtype=float)
        distance_matrix[idx] = np.pad(line_data, (0, labels.shape[0]- line_data.shape[0]), 'constant')
        
print(labels)
print(distance_matrix)

['Man' 'Monkey' 'Dog' 'Horse' 'Donkey' 'Pig' 'Rabbit']
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.]
 [13. 12.  0.  0.  0.  0.  0.]
 [17. 16. 10.  0.  0.  0.  0.]
 [16. 15.  8.  1.  0.  0.  0.]
 [13. 12.  4.  5.  4.  0.  0.]
 [12. 11.  6. 11. 10.  6.  0.]]


In [29]:
while(distance_matrix.shape[0] > 1):
    # find min
    temp_distance_matrix = distance_matrix.copy()
    temp_distance_matrix[temp_distance_matrix == 0] = np.inf
    min_position = np.unravel_index(np.argmin(temp_distance_matrix, axis=None), temp_distance_matrix.shape)
    
    # join
    new_distance_matrix = np.zeros((distance_matrix.shape[0]-1, distance_matrix.shape[0]-1), dtype=float)
    sum_of_first = np.sum([distance_matrix[min_position[1],:], distance_matrix[:,min_position[1]]], axis=0)
    sum_of_second = np.sum([distance_matrix[:,min_position[0]], distance_matrix[min_position[0],:]], axis=0)
    mean_of_union = np.mean([sum_of_first, sum_of_second], axis=0)
    mean_of_union[min_position[1]] = 0.0
    mean_of_union = np.delete(mean_of_union, min_position[0], 0)
    
    print('(' + labels[min_position[1]] + ', ' + labels[min_position[0]] + ')')
    
    labels = np.concatenate((labels[0:min_position[1]], ['(' + labels[min_position[1]] + ', ' + labels[min_position[0]] + ')'], 
                             labels[min_position[1]+1:min_position[0]], labels[min_position[0]+1:]), axis=0)
    
    distance_matrix = np.delete(distance_matrix, min_position[0], 0)
    distance_matrix = np.delete(distance_matrix, min_position[0], 1)
    
    distance_matrix[:,min_position[1]] = mean_of_union
    distance_matrix[min_position[1],:] = mean_of_union
    distance_matrix[np.triu_indices(distance_matrix.shape[1])] = 0

(Man, Monkey)
[[ 0.   0.   0.   0.   0.   0. ]
 [12.5  0.   0.   0.   0.   0. ]
 [16.5 10.   0.   0.   0.   0. ]
 [15.5  8.   1.   0.   0.   0. ]
 [12.5  4.   5.   4.   0.   0. ]
 [11.5  6.  11.  10.   6.   0. ]]
(Horse, Donkey)
[[ 0.   0.   0.   0.   0. ]
 [12.5  0.   0.   0.   0. ]
 [16.   9.   0.   0.   0. ]
 [12.5  4.   4.5  0.   0. ]
 [11.5  6.  10.5  6.   0. ]]
(Dog, Pig)
[[ 0.    0.    0.    0.  ]
 [12.5   0.    0.    0.  ]
 [16.    6.75  0.    0.  ]
 [11.5   6.   10.5   0.  ]]
((Dog, Pig), Rabbit)
[[ 0.     0.     0.   ]
 [12.     0.     0.   ]
 [16.     8.625  0.   ]]
(((Dog, Pig), Rabbit), (Horse, Donkey))
[[ 0.  0.]
 [14.  0.]]
((Man, Monkey), (((Dog, Pig), Rabbit), (Horse, Donkey)))
[[0.]]
